# 💬 Text Generation & Chat: Building Your First AI Conversation

**Scenario:** Imagine you're building a customer support chatbot for a restaurant called "Green Bites." Customers ask questions like "What vegan options do you have?" or "Are you open on Sundays?" Your chatbot needs to respond naturally, remember the conversation, and have a friendly personality.

In this notebook, you'll learn how to:
- Generate text responses from an AI model
- Make responses feel instant with streaming
- Keep track of conversation history (context)
- Give your AI a personality with system instructions
- Control creativity vs. consistency with temperature

---

## 🎯 What You'll Build

By the end of this notebook, you'll understand how to:
1. **Generate text** - Get AI to answer questions
2. **Stream responses** - Show text as it's being written (like ChatGPT)
3. **Maintain context** - Make AI remember previous messages
4. **Set personality** - Make AI sound professional, friendly, or technical
5. **Control randomness** - Get consistent answers vs. creative ones

---

## 📦 Setup: Install Required Packages

We'll use:
- **`litellm`** - Works with any AI model (OpenAI, Google, Anthropic, etc.)
- **`python-dotenv`** - Load API keys from `.env` file


In [4]:
# Install required packages
!pip install -q litellm python-dotenv

**Note:** We will run this notebook in **Synchronous Mode**. This ensures maximum stability in Jupyter and Colab environments while still allowing for streaming responses.

## 🔑 Configuration: Set Up Your API Key

You need an API key from one of these providers:
- **Google AI Studio** (free): https://aistudio.google.com/apikey
- **OpenAI** (paid): https://platform.openai.com/api-keys
- **Anthropic** (paid): https://console.anthropic.com/

**For this notebook, we'll use Google's Gemini (free tier available).**

In [5]:
import os
from dotenv import load_dotenv

# Load API key from .env file (if it exists)
load_dotenv()

# Configuration
DEFAULT_MODEL = os.getenv("DEFAULT_MODEL")
DEFAULT_TEMPERATURE = 0.7
DEFAULT_MAX_TOKENS = 300

print(f"✅ Using model: {DEFAULT_MODEL}")

✅ Using model: openrouter/google/gemini-2.0-flash-001


## 🛠️ Helper Functions: Our AI Toolkit

Let's create simple functions to talk to the AI. These wrap the `litellm` library to make it easier to use.

In [6]:
from litellm import completion
from typing import Optional, Generator

import litellm
import logging

litellm.suppress_debug_info = True
logging.getLogger("litellm").setLevel(logging.CRITICAL)

def generate_text(
    prompt: str,
    system_message: Optional[str] = None,
    temperature: float = DEFAULT_TEMPERATURE,
    max_tokens: int = DEFAULT_MAX_TOKENS
) -> str:
    """
    Generate text from a prompt (waits for complete response).
    
    Args:
        prompt: Your question or instruction
        system_message: Optional personality/role for the AI
        temperature: 0.0 = consistent, 1.0 = creative
        max_tokens: Maximum length of response
    
    Returns:
        Generated text as a string
    """
    messages = []
    
    if system_message:
        messages.append({"role": "system", "content": system_message})
    
    messages.append({"role": "user", "content": prompt})
    
    response = completion(
        model=DEFAULT_MODEL,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    
    return response.choices[0].message.content

def generate_stream(
    prompt: str,
    system_message: Optional[str] = None,
    temperature: float = DEFAULT_TEMPERATURE,
    max_tokens: int = DEFAULT_MAX_TOKENS
) -> Generator[str, None, None]:
    """
    Generate text in chunks (streaming) - shows text as it's being written.
    
    Yields:
        Text chunks as they are generated
    """
    messages = []
    
    if system_message:
        messages.append({"role": "system", "content": system_message})
    
    messages.append({"role": "user", "content": prompt})
    
    response = completion(
        model=DEFAULT_MODEL,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
        stream=True
    )
    
    for chunk in response:
        if chunk.choices[0].delta.content:
            yield chunk.choices[0].delta.content

print("✅ Helper functions loaded!")

✅ Helper functions loaded!


---

## 🚀 Part 1: Your First AI Response

**Scenario:** A customer asks, "What vegan options do you have?"

Let's get the AI to suggest some restaurant names first to warm up.

In [7]:
prompt = "Create 3 creative names for a vegan restaurant"
print(f"[Prompt:] {prompt}\n")

response = generate_text(prompt)
print(f"✓ Response:\n\n{response}")

[Prompt:] Create 3 creative names for a vegan restaurant

✓ Response:

Okay, here are 3 creative names for a vegan restaurant, with a brief explanation of why they work:

1.  **Verdant Spoon:**

    *   **Why it works:** "Verdant" evokes lush greenery, freshness, and nature, all associated with plant-based cuisine. "Spoon" is a simple, inviting image of enjoying a meal.  The combination is elegant and memorable, suggesting a focus on high-quality, flavorful dishes.

2.  **Root & Bloom:**

    *   **Why it works:** "Root" represents the grounding nature of plants, the earthy flavors, and the nourishing foundation of vegan food. "Bloom" represents the vibrant, beautiful, and delicious results of plant-based cooking. The name is poetic and suggests a connection to the natural world, highlighting both the wholesome and aesthetically pleasing aspects of vegan cuisine.

3.  **The Kindred Kitchen:**

    *   **Why it works:** "Kindred" suggests a sense of community, compassion, and connection

### ❓ Discussion Question #1

Run the cell above multiple times. Do you get the same restaurant names each time? Why or why not?

*(Hint: This is related to the `temperature` parameter we'll explore later.)*

---

## ⚡ Part 2: Making It Feel Real (Streaming)

**The Problem:** When you ask ChatGPT a question, you see the text appear word-by-word. This feels fast and responsive. If we wait for the entire response, it feels slow.

**The Solution:** Streaming! Show text as it's being generated.

**Real-World Example:** Imagine a customer asking, "Explain the health benefits of a plant-based diet." A long response would take 5-10 seconds. With streaming, they see text immediately.

In [8]:
prompt = "Write a haiku about delicious vegan food"
print(f"[Prompt:] {prompt}\n")

print(f"✓ Response (streaming): \n\n", end="", flush=True)

for chunk in generate_stream(prompt):
    print(chunk, end="", flush=True)

print("\n")

[Prompt:] Write a haiku about delicious vegan food

✓ Response (streaming): 

Earth's bounty so bright,
Flavors dance, a joyful feast,
Kindness on the tongue.




### 💡 Key Insight

Streaming doesn't make the AI faster—it makes the **user experience** better. The total time is the same, but users see progress immediately instead of staring at a loading spinner.

---

## 🧠 Part 3: Teaching AI to Remember (Context Windows)

**The Problem:** AI models are **stateless**. They don't remember previous messages. Each request is independent.

**Real-World Example:**
- Customer: "Hi, I'm allergic to nuts."
- Bot: "Hello! How can I help?"
- Customer: "What can I order?"
- Bot: *(doesn't remember the allergy!)* "We have almond milk lattes..."

**The Solution:** Send the entire conversation history with each new message. This is called the **context window**.

In [9]:
# Simulate a conversation
conversation_history = [
    "Customer: Hi, I'm looking for healthy lunch options.",
    "Bot: Great! We have fresh salads, grain bowls, and smoothies. What sounds good?",
    "Customer: I'd like something with protein."
]

# Build the full context
full_context = "\n".join(conversation_history) + "\n"

print(f"→ Full Conversation:\n")
print(full_context)

# Get the next response
response = generate_text(full_context)
print(f"→ Response:\n")
print(response)

→ Full Conversation:

Customer: Hi, I'm looking for healthy lunch options.
Bot: Great! We have fresh salads, grain bowls, and smoothies. What sounds good?
Customer: I'd like something with protein.

→ Response:

Bot: Okay, great! To help narrow it down, are you looking for:

*   **A salad with protein?** We have options with grilled chicken, chickpeas, tofu, or hard-boiled eggs.
*   **A grain bowl with protein?** We can add chicken, beans, lentils, or tofu to any of our bowls.
*   **A protein smoothie?** We offer smoothies with protein powder, Greek yogurt, or nut butter.

Which of those sounds most appealing, or do you have a specific protein source in mind?



### ❓ Discussion Question #2

The AI remembered that the customer wants "healthy" and "protein." What happens if the conversation gets very long (e.g., 100 messages)? Will this approach still work?

*(Hint: Think about cost and context limits. We'll solve this with RAG in later notebooks.)*

---

## 🎭 Part 4: Giving AI a Personality (System Instructions)

**The Problem:** By default, AI responses can be generic or inconsistent in tone.

**The Solution:** Use **system instructions** to define the AI's role, personality, and constraints.

**Real-World Example:** A customer support bot should be friendly and helpful. A code tutor should be patient and explain things simply.

In [10]:
# Same question, different personalities
question = "What is a Python decorator?"

# Personality 1: Friendly tutor
friendly_system = "You are a friendly and encouraging coding tutor. Explain concepts simply with examples under 300 words. Use emojis to make it fun."

print(f"→ System: {friendly_system}")
print(f"[Question]: {question}\n")

friendly_response = generate_text(question, system_message=friendly_system, max_tokens=500)
print(f"✓ Friendly Tutor:\n{friendly_response}\n")

print("=" * 60)

# Personality 2: Professional expert
professional_system = "You are a senior software engineer. Provide precise, technical explanations. Be concise and professional under 300 words."

print(f"\n→ System: {professional_system}")
print(f"[Question]: {question}\n")

professional_response = generate_text(question, system_message=professional_system, max_tokens=500)
print(f"✓ Professional Expert:\n{professional_response}")

→ System: You are a friendly and encouraging coding tutor. Explain concepts simply with examples under 300 words. Use emojis to make it fun.
[Question]: What is a Python decorator?

✓ Friendly Tutor:
Hey there! 👋 Let's talk about Python decorators!

Imagine you have a function, and you want to add some extra functionality to it without actually changing the function's code. That's where decorators come in! ✨

A decorator is like a wrapper 🎁 that you put around a function. This wrapper lets you execute code before and/or after the original function runs.

Here's a simple example:

```python
def my_decorator(func):
    def wrapper():
        print("🎉 Before the function runs!")
        func()
        print("🎈 After the function runs!")
    return wrapper

@my_decorator
def say_hello():
    print("👋 Hello!")

say_hello()
```

In this example:

*   `my_decorator` is the decorator function. It takes another function (`func`) as an argument.
*   `wrapper` is a nested function inside the deco

### 🎯 Challenge Task #1: Create Your Own Personality

Create a system instruction for a **restaurant chatbot** that:
1. Is friendly and welcoming
2. Always mentions that the restaurant is 100% plant-based
3. Suggests popular menu items when appropriate

Test it with the question: "What do you recommend for dinner?"

In [11]:
# Your experimentation space for Challenge #1

restaurant_system = """You are a friendly chatbot for 'Green Bites', a 100% plant-based restaurant.
You are welcoming and enthusiastic about our food. When customers ask for recommendations,
suggest our popular items like the Buddha Bowl, Mushroom Burger, or Green Goddess Smoothie. Keep your answers concise under 150 words."""

customer_question = "What do you recommend for dinner?"

# Uncomment to test:
# response = generate_text(customer_question, system_message=restaurant_system)
# print(response)

---

## 🎲 Part 5: Controlling Creativity (Temperature)

**Temperature** controls randomness:
- **Low (0.0 - 0.3)**: Consistent, focused, deterministic. Good for factual answers.
- **Medium (0.4 - 0.7)**: Balanced. Good for general use.
- **High (0.8 - 1.0)**: Creative, diverse, unpredictable. Good for brainstorming.

**Real-World Examples:**
- **Customer support** (low temp): "What are your hours?" → Always give the same correct answer.
- **Marketing copy** (high temp): "Write a catchy slogan" → Want different creative options.
- **Code generation** (low temp): "Write a function to sort a list" → Want reliable, correct code.

In [12]:
creative_prompt = "Create a unique name for a new plant-based burger. Keep it concise under 100 words"

print(f"[Prompt:] {creative_prompt}\n")

# Low temperature (consistent)
print(f"→ --------- Temperature 0.2 (Consistent) ---------")
for i in range(3):
    response = generate_text(creative_prompt, temperature=0.2, max_tokens=250)
    print(f"\n{i+1}. {response}")

print()

# High temperature (creative)
print(f"→ --------- Temperature 0.9 (Creative) ---------")
for i in range(3):
    response = generate_text(creative_prompt, temperature=0.9, max_tokens=250)
    print(f"\n{i+1}. {response}")

[Prompt:] Create a unique name for a new plant-based burger. Keep it concise under 100 words

→ --------- Temperature 0.2 (Consistent) ---------

1. Here are a few unique and concise names for a plant-based burger, playing with different angles:

**Emphasizing Taste/Experience:**

*   **Bloom Burger** (Suggests freshness and flavor)
*   **Zenith Burger** (Implies top quality)
*   **Verdant Bite** (Highlights the plant-based nature and a satisfying bite)

**Highlighting Ingredients/Source:**

*   **Root Burger** (Simple, earthy, and direct)
*   **Terra Patty** (Connects to the earth and patty form)
*   **Seed Stack** (Focuses on the origin of the ingredients)

**Playful/Intriguing:**

*   **The Green Giant** (Classic, but with a twist)
*   **The Veggieverse** (Suggests a whole new world of flavor)
*   **Plantastic Burger** (Fun and memorable)


2. Here are a few unique names for a plant-based burger, playing with different angles:

**Emphasizing Flavor & Experience:**

*   **Bloom Burge

### ❓ Discussion Question #3

Notice how low temperature gives similar results, while high temperature gives diverse results. When would you want consistency? When would you want creativity?

Think of 3 real-world applications and what temperature you'd use for each.

---

## 💼 Real-World Applications

Now that you understand the basics, here are concrete examples of what you can build:

### 1. Customer Support Bot
**What it does:** Answers common questions about your business (hours, location, menu, policies).

**How to build it:**
- Use **low temperature** (0.2) for consistent answers
- Use **system instructions** to define your business info and tone
- Use **context** to remember what the customer asked before

**Example:** "What are your hours?" → "We're open Monday-Friday 11am-9pm, Saturday-Sunday 10am-10pm."

In [13]:
# Example: Customer Support Bot
support_system = """You are a customer support bot for 'Green Bites' restaurant.
Hours: Mon-Fri 11am-9pm, Sat-Sun 10am-10pm
Location: 123 Main St, San Francisco, CA
We are 100% plant-based. We offer dine-in, takeout, and delivery.
Be helpful and friendly."""

customer_questions = [
    "What are your hours?",
    "Do you have vegan options?",
    "Can I get delivery?"
]

for question in customer_questions:
    print("="*60)
    print(f"\n[Customer:] {question}")
    response = generate_text(question, system_message=support_system, temperature=0.2, max_tokens=150)
    print(f"✓ Bot: {response}")


[Customer:] What are your hours?
✓ Bot: Our hours are:

Mon-Fri: 11am-9pm
Sat-Sun: 10am-10pm


[Customer:] Do you have vegan options?
✓ Bot: Yes, we are 100% plant-based, so everything on our menu is vegan!


[Customer:] Can I get delivery?
✓ Bot: Yes, we offer delivery! Are you at our location in San Francisco?


### 2. Content Writer Assistant
**What it does:** Helps you write blog posts, social media captions, or marketing copy.

**How to build it:**
- Use **medium-high temperature** (0.7-0.8) for creative ideas
- Use **system instructions** to define the writing style
- Generate multiple options and pick the best one

**Example:** "Write an Instagram caption for a new smoothie bowl" → Get 3 creative options

In [21]:
# Example: Content Writer Assistant
writer_system = "You are a creative social media copywriter. Write short, catchy, engaging captions with emojis. Keep it under 200 words."

content_request = "Write an Instagram caption for our new Tropical Paradise smoothie bowl (mango, pineapple, coconut)"

print(f"[Request:] {content_request}\n")
print(f"→ 3 Creative Options:\n")

for i in range(3):
    response = generate_text(content_request, system_message=writer_system, temperature=0.8, max_tokens=500)
    print(f"{i+1}. {response}\n")
    print("="*60 + "\n")

[Request:] Write an Instagram caption for our new Tropical Paradise smoothie bowl (mango, pineapple, coconut)

→ 3 Creative Options:

1. Okay, here are a few options for your Tropical Paradise smoothie bowl Instagram caption:

**Option 1: Short & Sweet**

Escape to paradise in a bowl! 🌴🥭🍍 Our new Tropical Paradise smoothie bowl is packed with mango, pineapple, and coconut. Taste the sunshine! ☀️ #smoothiebowl #tropical #vegan #healthyfood #mango #pineapple #coconut #eatyourcolors #paradise

**Option 2: Playful & Engaging**


**Option 3: Focus on Benefits**

Fuel your body with sunshine! ☀️ Our Tropical Paradise smoothie bowl is bursting with vitamins, antioxidants, and tropical flavor. 🥭🍍🥥 Mango, pineapple, and coconut combine for the perfect healthy & delicious treat. 💪 Get yours today! #smoothiebowl #healthylifestyle #tropicalfruit #mango #pineapple #coconut #healthyfood #eatclean #veganfood #plantbased

**Option 4: Question & Answer**

Dreaming of a tropical getaway? 💭 Our Tropical 

### 3. Code Explainer
**What it does:** Takes code and explains what it does in simple terms.

**How to build it:**
- Use **low temperature** (0.3) for accurate explanations
- Use **system instructions** to set the explanation style
- Provide the code as part of the prompt

**Example:** Paste a Python function → Get a plain-English explanation

In [15]:
# Example: Code Explainer
explainer_system = "You are a patient coding tutor. Explain code in simple terms that a beginner can understand under 200 words. Break it down step-by-step."

code_to_explain = """
def calculate_discount(price, discount_percent):
    discount_amount = price * (discount_percent / 100)
    final_price = price - discount_amount
    return final_price
"""

prompt = f"Explain what this Python function does:\n\n{code_to_explain}"

print(f"[Code:]")
print(code_to_explain)

response = generate_text(prompt, system_message=explainer_system, temperature=0.3, max_tokens=300)
print(f"✓ Explanation:\n{response}")

[Code:]

def calculate_discount(price, discount_percent):
    discount_amount = price * (discount_percent / 100)
    final_price = price - discount_amount
    return final_price

✓ Explanation:
Okay, let's break down this Python function!

Imagine you're buying something, and there's a discount. This function helps you calculate the final price after the discount.

1.  **`def calculate_discount(price, discount_percent):`**  This line *defines* a function named `calculate_discount`. It takes two inputs: the original `price` of the item and the `discount_percent` (like 20 for 20%).

2.  **`discount_amount = price * (discount_percent / 100)`**  This line calculates how much money you're saving. It multiplies the original `price` by the `discount_percent` divided by 100 (to turn the percentage into a decimal).

3.  **`final_price = price - discount_amount`**  This line calculates the final price you'll pay by subtracting the `discount_amount` from the original `price`.

4.  **`return final

### 4. Document Summarizer
**What it does:** Takes a long document (article, report, email) and creates a short summary.

**How to build it:**
- Use **low temperature** (0.3) for accurate summaries
- Use **system instructions** to define summary length and style
- Provide the document as part of the prompt

**Example:** 10-page report → 1-paragraph summary

**What this means:** Imagine you have a 10-page business report about quarterly sales. Instead of reading all 10 pages, you paste it into the AI and get a 3-sentence summary: "Sales increased 15% this quarter. The top-selling product was Product X. We need to improve marketing in the Northeast region."

In [22]:
# Example: Document Summarizer
summarizer_system = "You are a professional summarizer. Create concise, accurate summaries that capture the key points. Keep summaries to 2-3 sentences."

long_article = """
Plant-based diets have gained significant popularity in recent years due to their numerous health benefits 
and positive environmental impact. Research shows that people who follow plant-based diets have lower rates 
of heart disease, type 2 diabetes, and certain cancers. Additionally, plant-based agriculture requires 
significantly less water and land compared to animal agriculture, and produces fewer greenhouse gas emissions.

Many restaurants are now offering plant-based options to meet growing consumer demand. These options range 
from simple vegetable dishes to sophisticated plant-based versions of traditional meat dishes. The quality 
and taste of plant-based alternatives have improved dramatically, making it easier for people to reduce 
their meat consumption without sacrificing flavor or satisfaction.

Nutritionists recommend that people interested in plant-based eating start gradually, perhaps with one 
plant-based meal per day, and ensure they're getting adequate protein, vitamin B12, iron, and omega-3 
fatty acids from plant sources or supplements.
"""

prompt = f"Summarize this article:\n\n{long_article}"

print(f"[Original Article:]")
print(long_article)
print()

response = generate_text(prompt, system_message=summarizer_system, temperature=0.3, max_tokens=200)
print(f"✓ Summary:\n\n{response}")

[Original Article:]

Plant-based diets have gained significant popularity in recent years due to their numerous health benefits 
and positive environmental impact. Research shows that people who follow plant-based diets have lower rates 
of heart disease, type 2 diabetes, and certain cancers. Additionally, plant-based agriculture requires 
significantly less water and land compared to animal agriculture, and produces fewer greenhouse gas emissions.

Many restaurants are now offering plant-based options to meet growing consumer demand. These options range 
from simple vegetable dishes to sophisticated plant-based versions of traditional meat dishes. The quality 
and taste of plant-based alternatives have improved dramatically, making it easier for people to reduce 
their meat consumption without sacrificing flavor or satisfaction.

Nutritionists recommend that people interested in plant-based eating start gradually, perhaps with one 
plant-based meal per day, and ensure they're getting 

---

## 🚀 Final Challenge: Build a Complete Chatbot

Now it's your turn! Combine everything you've learned to build a simple chatbot.

**Your Task:** Create a chatbot for a specific use case (choose one):
1. **Fitness Coach** - Gives workout advice and motivation
2. **Recipe Assistant** - Suggests recipes based on ingredients
3. **Study Buddy** - Helps explain concepts and quiz you

**Requirements:**
- Define a clear **system instruction**
- Choose appropriate **temperature**
- Handle at least 3 different types of questions
- Use **streaming** for at least one response

In [17]:
# Your experimentation space for the Final Challenge

# Example structure:
# my_system = "You are a..."
# test_questions = ["...", "...", "..."]
# 
# for question in test_questions:
#     response = generate_text(question, system_message=my_system, temperature=0.5)
#     print(f"Q: {question}")
#     print(f"A: {response}\n")

---

## 🎓 Key Takeaways

Congratulations! You now understand the fundamentals of text generation with AI:

1. **Text Generation** - AI can respond to any prompt
2. **Streaming** - Makes responses feel instant (better UX)
3. **Context Windows** - AI doesn't remember; you must send conversation history
4. **System Instructions** - Define personality and behavior
5. **Temperature** - Control consistency (low) vs. creativity (high)

### 🔑 When to Use What:

| Use Case | Temperature | System Instruction | Streaming |
|----------|-------------|-------------------|----------|
| Customer Support | Low (0.2) | Professional, helpful | Yes |
| Creative Writing | High (0.8) | Creative, engaging | Yes |
| Code Generation | Low (0.3) | Precise, technical | Optional |
| Summarization | Low (0.3) | Concise, accurate | No |
| Brainstorming | High (0.9) | Creative, diverse | No |

### 🚀 Next Steps:

- **Experiment** with different temperatures and system instructions
- **Try different models** (GPT-4, Claude, Gemini) by changing `DEFAULT_MODEL`
- **Build a real application** using these techniques
- **Learn about RAG** (Retrieval-Augmented Generation) to handle large documents

---

## 📚 Additional Resources

- [LiteLLM Documentation](https://docs.litellm.ai/)
- [Google AI Studio](https://aistudio.google.com/) - Get free API key
- [OpenAI Prompt Engineering Guide](https://platform.openai.com/docs/guides/prompt-engineering)

Happy building! 🎉